In [14]:
import pandas as pd
import numpy as np
import os
from subprocess import call
from tqdm import tqdm
import datetime
import pymongo
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

In [2]:
df = pd.read_csv("./india_info.csv")
india_info = df[['state', 'city']]

In [3]:
india_info

state        city
0    Maharashtra      Mumbai
1          Delhi       Delhi
2      Karnataka   Bangalore
3      Telangana   Hyderabad
4        Gujarat   Ahmedabad
..           ...         ...
304        Bihar       Buxar
305        Assam      Tezpur
306        Bihar   Jehanabad
307        Bihar  Aurangabad
308       Sikkim     Gangtok

[309 rows x 2 columns]

In [4]:
bash_command_list = []

for i, row in india_info.iterrows():
    state, city = row['state'], row['city']
    temp = "twint -s 'Unlock1 AND {}' --limit 1 -o 'output/{}--{}.csv' --csv --since '2020-7-1 00:00:00'".format(city, state, city)
    bash_command_list.append(temp)

In [5]:
file1 = open("bash_command_list.txt","w")
file1.writelines(bash_command_list) 
file1.close()

In [6]:
bash_command_list[0]

"twint -s 'Unlock1 AND Mumbai' --limit 1 -o 'output/Maharashtra--Mumbai.csv' --csv --since '2020-7-1 00:00:00'"

In [30]:
for bash_command in bash_command_list[:5]:
    call(bash_command, shell=True)

In [7]:
scrapped_data = os.listdir("./output/")

In [8]:
scrapped_data

['Delhi--Delhi.csv',
 'Gujarat--Ahmedabad.csv',
 'Karnataka--Bangalore.csv',
 'Maharashtra--Mumbai.csv',
 'Telangana--Hyderabad.csv']

In [9]:
output_df = pd.DataFrame()

for file in scrapped_data:
    file_df = pd.read_csv('output/{}'.format(file))
    
    temp = file.split(".")[0].split("--")
    state, city = temp[0], temp[1]
    
    file_df['state'] = state
    file_df['city'] = city
    
    output_df = output_df.append(file_df)

In [10]:
output_df = output_df.reset_index()

In [11]:
output_df.columns

Index(['index', 'id', 'conversation_id', 'created_at', 'date', 'time',
       'timezone', 'user_id', 'username', 'name', 'place', 'tweet', 'mentions',
       'urls', 'photos', 'replies_count', 'retweets_count', 'likes_count',
       'hashtags', 'cashtags', 'link', 'retweet', 'quote_url', 'video', 'near',
       'geo', 'source', 'user_rt_id', 'user_rt', 'retweet_id', 'reply_to',
       'retweet_date', 'translate', 'trans_src', 'trans_dest', 'state',
       'city'],
      dtype='object')

In [12]:
df_to_work_with = output_df[['tweet', 'link', 'date', 'time', 'state', 'city']]

In [15]:
df_to_work_with

tweet  \
0   That’s when d role of govts. B’comes v. releva...   
1   - Delhi trying to chase numbers - #politics in...   
2   सरकार कुछ समय के लिए ही आपकी मदद कर सकती है। ब...   
3   Even Delhi has a software to find out where be...   
4   ਜਾਣੋ, ਕੋਰੋਨਾ ਨੂੰ ਲੈ ਕੇ ਕੀ ਹੈ ਭਾਰਤ ਦੀ ਸਥਿਤੀ ?\n...   
5   India has recorded over 6.25 lakh cases of #co...   
6   #Delhi #Coronavirus Update (Data of 2 July 202...   
7   AUTO DRIVER/ऑटो ड्राईवर ने COVID-19 में ऑटो/AU...   
8   #Delhi #coronavirus Update (Data of 1 July 202...   
9   Just thinking 🤔 #1stJuly #Delhi #lockdown #Unl...   
10  2)The situation is improving in #Delhi in the ...   
11  LEAD with DIKSHA an initiative by SCERT Delhi ...   
12  અનલોક-2: નડિયાદનું સંતરામ મંદિર ભક્તો માટે બંધ...   
13  #Bangalore has become a victim of #unlock1. Im...   
14  It has started happening in Bangalore which wa...   
15  Yeah even I'm from Karnataka at the time of Ju...   
16  Halo! the unroll you asked for: @covid19indiao...   
17  The shifting case share from Mumbai to other d...   
18  I think it’s as @vnehra said #NIMBY EFFECT ( n...   
19   https://www.cnbctv18.com/photos/travel/in-pic...   
20  #Covid19 #India #Lockdown #Unlock1 #Unlock2 #J...   
21  Craving for a hot bowl of #Ramen? Order away!\...   
22  पुण्यात महापालिकेकडून 'या' पदासाठी मोठी भरती स...   
23  विद्यापीठ परीक्षांबाबतीच राज्यसरकार घेऊ शकते म...   
24  खताच्या मागणीसाठी लागल्या लांबच लांब रांगा\n\n...   
25  Maharashtra Rain Update | पाहा आज महाराष्ट्रात...   
26  गूड न्यूज! राज्यात पोलीस शिपाई संवर्गात होणार ...   
27  भर लॉकडाऊनमध्ये आंतराष्ट्रीय क्रिकेट उद्यापासू...   
28  Mr.UT if u r so confident of Mumbai's covid si...   
29  बर्थडे बॉय रणवीर सिंगचे हे अतरंगी लूक पाहिले आ...   
30  येत्या तीन दिवसात मुंबईत पुन्हा मुसळधार पावसाच...   
31  राज्यातील मृत्यूंमध्ये 73 % टक्के मृत्यू हे MM...   
32  Lokshahi Special | देवेंद्र फडणवीसांचं बोलणं क...   
33  गाडी चालवताना बुलेटच्या सीटखालून निघाला साप आण...   
34  The only reason why #COVID19 infected cases ar...   
35  मुंबईतल्या वार्डनुसार पाहा कोरोनाचे कुठे आणि क...   
36  .@starlingpayal doing the thing she loves the ...   
37  .@LakshmiManchu tells us all about her #LifeIn...   
38  hawa me dhua chha raha he,\nlockdown khatam ho...   
39  .@urstrulyMahesh says #Krishna's smile is what...   
40  .@urstrulyMahesh's birthday wishes for his mak...   
41  Just @shraddhadas43, chilling at home in a sar...   
42  Haha! @Itslavanya keeping things light in #qua...   
43  @Jksoniias @NagaurPolice \nAround 60% populati...   
44  .@bhumipednekar on a 'watermelon sugar high' a...   
45  Just @Itslavanya pouting away😚😍\n#LavanyaTripa...   
46  It's mean, no more lockdowns in #Hyderabad \n\...   
47  Check out the Hyderabad Times e-paper:  http:/...   
48  Just @MukhiSree enjoying the sights she sees d...   
49  Check out the Hyderabad Times e-paper:  http:/...   
50  Check out the Hyderabad Times e-paper:  http:/...   

                                                 link        date      time  \
0   https://twitter.com/DrLahariya/status/12814632...  2020-07-10  11:10:16   
1   https://twitter.com/sarungss/status/1281128386...  2020-07-09  12:59:44   
2   https://twitter.com/im_delhi/status/1280171474...  2020-07-06  21:37:18   
3   https://twitter.com/praveennayak/status/127936...  2020-07-04  15:58:58   
4   https://twitter.com/ptcnews/status/12793229729...  2020-07-04  13:25:40   
5   https://twitter.com/moneycontrolcom/status/127...  2020-07-03  10:09:26   
6   https://twitter.com/BugleInstant/status/127870...  2020-07-02  20:28:14   
7   https://twitter.com/laatsaabnews/status/127863...  2020-07-02  16:09:06   
8   https://twitter.com/BugleInstant/status/127835...  2020-07-01  21:20:47   
9   https://twitter.com/shree14667178/status/12783...  2020-07-01  20:50:17   
10  https://twitter.com/BugleInstant/status/127825...  2020-07-01  14:22:03   
11  https://twitter.com/EMC_Delhi/status/127819756...  2020-07-01  10:53:41   
12  https://twitter.com/revoiindia/status/12787121...  2

In [16]:
model = load_model("./model/model_v6.h5")
tokenizer = pickle.load(open("./model/tokenizer_v6.pickle", "rb"))

In [17]:
score_list = ["anger", "happiness", "neutral", "sadness", "worry"]

In [19]:
for i, row in df_to_work_with.iterrows():
    sequences = tokenizer.texts_to_sequences([row['tweet']])
    new_processed = pad_sequences(sequences, padding="post", maxlen=30)
    score = model.predict(new_processed)
    print(score_list[np.argmax(score)])

neutral
worry
neutral
worry
neutral
anger
worry
worry
worry
neutral
worry
neutral
neutral
neutral
neutral
sadness
neutral
worry
worry
neutral
neutral
neutral
worry
worry
worry
worry
worry
worry
anger
worry
worry
worry
sadness
worry
sadness
neutral
neutral
neutral
neutral
neutral
happiness
neutral
happiness
anger
neutral
neutral
neutral
neutral
neutral
neutral
neutral


In [20]:
def analyze(tweet):
    sequences = tokenizer.texts_to_sequences([row['tweet']])
    new_processed = pad_sequences(sequences, padding="post", maxlen=30)
    return(model.predict(new_processed))

In [21]:
analyze('this is so sad')

array([[0.00141264, 0.16456127, 0.6715156 , 0.0526654 , 0.10984511]],
      dtype=float32)

In [22]:
analyze('this is so sad this is so sad this is so sad this is so sad')

array([[0.00141264, 0.16456127, 0.6715156 , 0.0526654 , 0.10984511]],
      dtype=float32)

In [29]:
output = pd.DataFrame()

for index, row in tqdm(df_to_work_with[['date', 'time', 'tweet', 'link', 'state', 'city']].iterrows()):

    if(index % 20 == 0):
        print("At Index: {}".format(index))

    date = row['date']
    time = row['time']
    
    date_time = date + ' ' + time
    date_time = datetime.datetime.strptime(date_time, '%Y-%m-%d %H:%M:%S')
    
    tweet = row['tweet']
    link = row['link']
    state = row['state']
    city = row['city']

    score = analyze(tweet)
    
    anger = score[0][0]
    happiness = score[0][1]
    neutral = score[0][2]
    sadness = score[0][3]
    worry = score[0][4]
    
    result = {'date_time': date_time, 'tweet': tweet, 'link': link, 'state': state, 'city': city,
             'anger': anger, 'happiness': happiness, 'neutral': neutral, 'sadness': sadness, 'worry': worry }
    
    output = output.append(result, ignore_index=True)

1it [00:00,  7.94it/s]

At Index: 0


22it [00:01, 11.32it/s]

At Index: 20


42it [00:03, 12.04it/s]

At Index: 40


51it [00:04, 11.03it/s]


In [30]:
output[['date_time', 'tweet', 'link', 'state', 'city', 'anger', 'happiness', 'neutral' , 'sadness', 'worry']].to_csv('{}-backup.csv'.format(datetime.date.today()))
output[['date_time', 'tweet', 'link', 'state', 'city', 'anger', 'happiness', 'neutral' , 'sadness', 'worry']]

date_time                                              tweet  \
0  2020-07-10 11:10:16  That’s when d role of govts. B’comes v. releva...   
1  2020-07-09 12:59:44  - Delhi trying to chase numbers - #politics in...   
2  2020-07-06 21:37:18  सरकार कुछ समय के लिए ही आपकी मदद कर सकती है। ब...   
3  2020-07-04 15:58:58  Even Delhi has a software to find out where be...   
4  2020-07-04 13:25:40  ਜਾਣੋ, ਕੋਰੋਨਾ ਨੂੰ ਲੈ ਕੇ ਕੀ ਹੈ ਭਾਰਤ ਦੀ ਸਥਿਤੀ ?\n...   
5  2020-07-03 10:09:26  India has recorded over 6.25 lakh cases of #co...   
6  2020-07-02 20:28:14  #Delhi #Coronavirus Update (Data of 2 July 202...   
7  2020-07-02 16:09:06  AUTO DRIVER/ऑटो ड्राईवर ने COVID-19 में ऑटो/AU...   
8  2020-07-01 21:20:47  #Delhi #coronavirus Update (Data of 1 July 202...   
9  2020-07-01 20:50:17  Just thinking 🤔 #1stJuly #Delhi #lockdown #Unl...   
10 2020-07-01 14:22:03  2)The situation is improving in #Delhi in the ...   
11 2020-07-01 10:53:41  LEAD with DIKSHA an initiative by SCERT Delhi ...   
12 2020-07-02 20:58:20  અનલોક-2: નડિયાદનું સંતરામ મંદિર ભક્તો માટે બંધ...   
13 2020-07-09 10:33:24  #Bangalore has become a victim of #unlock1. Im...   
14 2020-07-06 07:56:44  It has started happening in Bangalore which wa...   
15 2020-07-05 19:46:42  Yeah even I'm from Karnataka at the time of Ju...   
16 2020-07-10 21:31:28  Halo! the unroll you asked for: @covid19indiao...   
17 2020-07-10 21:21:38  The shifting case share from Mumbai to other d...   
18 2020-07-09 21:04:05  I think it’s as @vnehra said #NIMBY EFFECT ( n...   
19 2020-07-09 18:05:00   https://www.cnbctv18.com/photos/travel/in-pic...   
20 2020-07-09 16:59:40  #Covid19 #India #Lockdown #Unlock1 #Unlock2 #J...   
21 2020-07-09 16:00:00  Craving for a hot bowl of #Ramen? Order away!\...   
22 2020-07-07 21:01:33  पुण्यात महापालिकेकडून 'या' पदासाठी मोठी भरती स...   
23 2020-07-07 20:16:29  विद्यापीठ परीक्षांबाबतीच राज्यसरकार घेऊ शकते म...   
24 2020-07-07 19:50:28  खताच्या मागणीसाठी लागल्या लांबच लांब रांगा\n\n...   
25 2020-07-07 19:41:44  Maharashtra Rain Update | पाहा आज महाराष्ट्रात...   
26 2020-07-07 19:09:37  गूड न्यूज! राज्यात पोलीस शिपाई संवर्गात होणार ...   
27 2020-07-07 17:12:04  भर लॉकडाऊनमध्ये आंतराष्ट्रीय क्रिकेट उद्यापासू...   
28 2020-07-06 19:58:12  Mr.UT if u r so confident of Mumbai's covid si...   
29 2020-07-06 19:30:56  बर्थडे बॉय रणवीर सिंगचे हे अतरंगी लूक पाहिले आ...   
30 2020-07-06 19:22:06  येत्या तीन दिवसात मुंबईत पुन्हा मुसळधार पावसाच...   
31 2020-07-06 19:10:28  राज्यातील मृत्यूंमध्ये 73 % टक्के मृत्यू हे MM...   
32 2020-07-06 18:54:13  Lokshahi Special | देवेंद्र फडणवीसांचं बोलणं क...   
33 2020-07-06 17:55:36  गाडी चालवताना बुलेटच्या सीटखालून निघाला साप आण...   
34 2020-07-05 22:34:26  The only reason why #COVID19 infected cases ar...   
35 2020-07-04 17:13:58  मुंबईतल्या वार्डनुसार पाहा कोरोनाचे कुठे आणि क...   
36 2020-07-10 12:15:00  .@starlingpayal doing the thing she loves the ...   
37 2020-07-10 10:15:00  .@LakshmiManchu tells us all about her #LifeIn...   
38 2020-07-09 19:07:04  hawa me dhua chha raha he,\nlockdown khatam ho...   
39 2020-07-09 12:54:16  .@urstrulyMahesh says #Krishna's smile is what...   
40 2020-07-08 13:36:07  .@urstrulyMahesh's birthday wishes for his mak...   
41 2020-07-06 13:00:00  Just @shraddhadas43, chilling at home in a sar...   
42 2020-07-06 12:15:00  Haha! @Itslavanya keeping things light in #qua...   
43 2020-07-05 21:09:08  @Jksoniias @NagaurPolice \nAround 60% populati...   
44 2020-07-05 13:15:00  .@bhumipednekar on a 'watermelon sugar high' a...   
45 2020-07-05 13:00:00  Just @Itslavanya pouting away😚😍\n#LavanyaTripa...   
46 2020-07-04 17:01:37  It's mean, no more lockdowns in #Hyderabad \n\...   
47 2020-07-03 08:16:12  Check out the Hyderabad Times e-paper:  http:/...   
48 2020-07-02 12:15:00  Just @MukhiSree enjoying the sights she sees d...   
49 2020-07-02 10:00:00  Check out the Hyderabad Times e-paper:  http:/...   
50 2020-07-01 10:00:00  Check out the Hyderabad Times e-paper:  http:/...   

        

In [75]:
mongo_url = os.environ.get("mongo_url")
myclient = pymongo.MongoClient(os.getenv("covidian_server_internal_mongo_url"))
mydb = myclient["covidian"]

# Collection: Sentiments

In [28]:
mycol = mydb["sentiments"]

In [31]:
df_for_sentiments = output[['date_time', 'tweet', 'link', 'state', 'city', 'anger', 'happiness', 'neutral' , 'sadness', 'worry']]

In [38]:
for i, row in df_for_sentiments[:1].iterrows():
    mycol.insert_one(dict(row))

# Collection: Sentiments_City

In [48]:
mycol = mydb["sentiments_city"]
df_for_sentiments_city = output[['date_time', 'tweet', 'link', 'state', 'city', 'anger', 'happiness', 'neutral' , 'sadness', 'worry']].groupby(['state', 'city'], as_index=False).mean()

In [50]:
df_for_sentiments_city

state       city     anger  happiness   neutral   sadness     worry
0        Delhi      Delhi  0.122905   0.089006  0.286426  0.169298  0.332365
1      Gujarat  Ahmedabad  0.028146   0.102312  0.381432  0.196105  0.292006
2    Karnataka  Bangalore  0.065285   0.106214  0.261186  0.295185  0.272129
3  Maharashtra     Mumbai  0.082140   0.107188  0.264237  0.206801  0.339634
4    Telangana  Hyderabad  0.032825   0.229237  0.468903  0.106881  0.162153

In [51]:
for i, row in df_for_sentiments_city[:1].iterrows():
    mycol.insert_one(dict(row))

# Collection: Sentiments_State

In [53]:
mycol = mydb["sentiments_state"]
df_for_sentiments_state = output[['date_time', 'tweet', 'link', 'state', 'city', 'anger', 'happiness', 'neutral' , 'sadness', 'worry']].groupby(['state'], as_index=False).mean()

In [54]:
df_for_sentiments_state

state     anger  happiness   neutral   sadness     worry
0        Delhi  0.122905   0.089006  0.286426  0.169298  0.332365
1      Gujarat  0.028146   0.102312  0.381432  0.196105  0.292006
2    Karnataka  0.065285   0.106214  0.261186  0.295185  0.272129
3  Maharashtra  0.082140   0.107188  0.264237  0.206801  0.339634
4    Telangana  0.032825   0.229237  0.468903  0.106881  0.162153

In [55]:
for i, row in df_for_sentiments_state[:1].iterrows():
    mycol.insert_one(dict(row))

# Collection: Sentiments_Country

In [57]:
mycol = mydb["sentiments_country"]
df_for_sentiments_country = output[['date_time', 'tweet', 'link', 'state', 'city', 'anger', 'happiness', 'neutral' , 'sadness', 'worry']].mean()

In [66]:
sentiments_country = dict()
sentiments_country['country'] = 'India'
sentiments_country.update(df_for_sentiments_country.to_dict())

In [67]:
sentiments_country

{'country': 'India',
 'anger': 0.0751771296958859,
 'happiness': 0.138653963804245,
 'neutral': 0.33177240061409335,
 'sadness': 0.17357787387628182,
 'worry': 0.2808186382639642}

In [68]:
mycol = mydb["sentiments_country"]
mycol.insert_one(sentiments_country)

In [69]:
from dotenv import load_dotenv
load_dotenv()

True

In [74]:
print(os.getenv("covidian_client_external_port"))

5000
